In [1]:
# pip install missingno
# pip install pandas
# pip install yahoo-finance
# pip install numpy
# pip install scikit-learn
# pip install kneed

In [2]:
import yahoo_fin.stock_info as si
import pandas as pd
import missingno
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn import metrics
import matplotlib.pyplot as plt
from kneed import KneeLocator
from sklearn.metrics import silhouette_score



In [3]:
# Extract the nasdaq data from the past 5 years
# Sometimes the code is not correctly working, I guess it's because of the API error. Try to re-run it again.
nasdaq_list = si.tickers_sp500()
nasdaq_historical = {}
for ticker in nasdaq_list:
    nasdaq_historical[ticker] = si.get_data(ticker, start_date="01/01/2019", index_as_date = False, interval="1d")
    print(ticker)
nasdaq_historical

A
AAL
AAP
AAPL
ABBV
ABC
ABMD
ABT
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALK
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
ATVI
AVB
AVGO
AVY
AWK
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BIIB
BIO
BK
BKNG
BKR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDAY
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COST
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DD
DE
DFS
DG
DGX
DHI
DHR
DIS
DISH
DLR
DLTR
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXC
DXCM
EA
EBAY
ECL
ED
EFX
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FBHS
FCX
FDS
FDX
FE
FFIV
FIS
FISV
FITB
FLT
FMC
FOX
FOXA
FRC
FRT
FTNT
FTV
GD
GE
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUM
HWM
IBM
ICE
IDXX
IEX
IFF
ILMN
INCY
INTC
INTU

{'A':           date        open        high         low       close    adjclose  \
 0   2019-01-02   66.500000   66.570000   65.300003   65.690002   63.937458   
 1   2019-01-03   65.529999   65.779999   62.000000   63.270000   61.582005   
 2   2019-01-04   64.089996   65.949997   64.089996   65.459999   63.713573   
 3   2019-01-07   65.639999   67.430000   65.610001   66.849998   65.066498   
 4   2019-01-08   67.589996   68.209999   66.699997   67.830002   66.020363   
 ..         ...         ...         ...         ...         ...         ...   
 959 2022-10-21  126.370003  130.130005  124.820000  129.880005  129.880005   
 960 2022-10-24  131.130005  133.130005  129.809998  132.300003  132.300003   
 961 2022-10-25  132.300003  135.100006  132.300003  134.500000  134.500000   
 962 2022-10-26  134.429993  139.309998  134.000000  137.690002  137.690002   
 963 2022-10-27  137.259995  137.990005  135.384995  136.690002  136.690002   
 
       volume ticker  
 0    2113300      A  

In [4]:
data = pd.concat(nasdaq_historical)
data.reset_index(drop=True, inplace=True)

In [5]:
# Drop every missing data
missing = data.isnull().mean().sort_values(ascending=False)
drop_list = sorted(list(missing[missing > 0.15].index))
data.drop(labels=drop_list, axis=1, inplace=True)

In [6]:
# fill with last data
data = data.fillna(method='ffill')
data

,date,open,high,low,close,adjclose,volume,ticker
0,2019-01-02,66.500000,66.570000,65.300003,65.690002,63.937458,2113300,A
1,2019-01-03,65.529999,65.779999,62.000000,63.270000,61.582005,5383900,A
2,2019-01-04,64.089996,65.949997,64.089996,65.459999,63.713573,3123700,A
3,2019-01-07,65.639999,67.430000,65.610001,66.849998,65.066498,3235100,A
4,2019-01-08,67.589996,68.209999,66.699997,67.830002,66.020363,1578100,A
...,...,...,...,...,...,...,...,...
482666,2022-10-21,143.940002,147.690002,142.119995,147.210007,147.210007,2354100,ZTS
482667,2022-10-24,148.710007,150.649994,148.320007,149.369995,149.369995,1356600,ZTS
482668,2022-10-25,149.259995,152.550003,149.259995,152.300003,152.300003,2065600,ZTS
482669,2022-10-26,152.190002,154.919998,151.130005,151.669998,151.669998,2162300,ZTS


In [7]:
# Start the calculation 
# 252 trading days in a year
returns = data.pct_change().mean()*252

returns = pd.DataFrame(returns)
returns.columns = ['returns']

returns['volatility'] = data.pct_change().std()*np.sqrt(252)
data = returns
data.head()

TypeError: cannot perform __truediv__ with this index type: DatetimeArray

In [ ]:
scale = StandardScaler().fit(data)

#Fit the scaler
scaled_data = pd.DataFrame(scale.fit_transform(data),columns = data.columns, index = data.index)
X = scaled_data

In [ ]:
# Use Elbow method to determine the K value
K = range(1,20)
distortions = []
for k in K:
    kmeans = KMeans(n_clusters = k)
    kmeans.fit(X)
    distortions.append(kmeans.inertia_)

fig = plt.figure(figsize= (30,10))
plt.plot(K, distortions, 'bx-')
plt.title('Elbow Method')
plt.show()

In [ ]:
# We know the K-value we should use
kl = KneeLocator(K, distortions, curve="convex", direction="decreasing")
# Save the number of clusters using b
a=kl.elbow
a

In [ ]:
# Use Silhouette method to determine the K value
K = range(2,20)
silhouettes = []
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10, init='random')
    kmeans.fit(X)
    silhouettes.append(silhouette_score(X, kmeans.labels_))
fig = plt.figure(figsize= (30,10))
plt.plot(K, silhouettes, 'bx-')
plt.title('Silhouette Method')
plt.show()

kl = KneeLocator(K, silhouettes, curve="convex", direction="decreasing")
# Save the number of clusters using b
b=kl.elbow
b

In [ ]:
# Apply the K-means model for the silhoutte
k_means = KMeans(n_clusters=a)
k_means.fit(X)
prediction = k_means.predict(X)

#Plot the results
centroids = k_means.cluster_centers_
fig = plt.figure(figsize = (15,8))
ax = fig.add_subplot(111)
scatter = ax.scatter(X.iloc[:,0],X.iloc[:,1], c=k_means.labels_, cmap="rainbow", label = X.index)
ax.set_title('k-Means Clustering')
ax.set_xlabel('Mean Return')
ax.set_ylabel('Volatility')
plt.colorbar(scatter)
plt.plot(centroids[:,0],centroids[:,1],'rs',markersize=8)
plt.show()

In [ ]:
# Apply the K-means model for the elbow
k_means = KMeans(n_clusters=b)
k_means.fit(X)
prediction = k_means.predict(X)

#Plot the results
centroids = k_means.cluster_centers_
ax = fig.add_subplot(111)
scatter = ax.scatter(X.iloc[:,0],X.iloc[:,1], c=k_means.labels_, cmap="rainbow", label = X.index)
ax.set_title('k-Means Clustering')
ax.set_xlabel('Mean Return')
ax.set_ylabel('Volatility')
plt.colorbar(scatter)
plt.plot(centroids[:,0],centroids[:,1],'rs',markersize=8)
plt.show()

In [ ]:
clustered_groups = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_groups_all = pd.Series(index=X.index, data=k_means.labels_.flatten())
clustered_groups = clustered_groups[clustered_groups != -1]
plt.barh(range(len(clustered_groups.value_counts())),clustered_groups.value_counts())
plt.title('Clustering results')
plt.xlabel('Number of stocks belong to the same group')
plt.ylabel('Cluster Index')
plt.show()